In [1]:
import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.retriever.embedder import Embedder

embedder = Embedder()
vec = embedder.encode("This is a test sentence.")
vec.shape

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model: all-MiniLM-L6-v2


(1, 384)

In [3]:
from src.retriever.vector_store import VectorStore

store = VectorStore.load(
    index_path="../models/faiss_index.bin",
    meta_path="../models/metadata.pkl"
)

Loaded index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)


In [4]:
embeder = Embedder()

query = "fantasy romance with magic and dragons"
q_vec = embedder.encode(query)[0]

results = store.search(q_vec, k=5)

Loading embedding model: all-MiniLM-L6-v2


In [5]:
for i, r in enumerate(results, start=1):
    meta = r["metadata"]
    print(f"\nResult {i}")
    print(f"Title:   {meta['title']}")
    print(f"Author:  {meta['author']}")
    print(f"Genres:  {meta['genres']}")
    print(f"Rating:  {meta['rating']}")
    print(f"Distance: {r['distance']:.4f}")


Result 1
Title:   The Girl the Dragon and the Wild Magic (Rhianna #1)
Author:  Dave Luckett
Genres:  fantasy, fantasy,dragons, fantasy,magic, childrens,middle grade, childrens, adventure, fantasy,high fantasy, fantasy,fairy tales, science fiction fantasy, kids
Rating:  3.86
Distance: 0.7392

Result 2
Title:   Must Love Dragons (Immortally Sexy #2)
Author:  Stephanie Rowe
Genres:  fantasy,paranormal, romance, romance,paranormal romance, fantasy,dragons, fantasy, fantasy,magic, paranormal,shapeshifters, humor, fantasy,supernatural, paranormal,demons
Rating:  3.98
Distance: 0.7410

Result 3
Title:   Master of Dragons (Mageverse #5)
Author:  Angela Knight
Genres:  romance,paranormal romance, fantasy,paranormal, romance, fantasy, fantasy,dragons, paranormal,vampires, fantasy,magic, paranormal,shapeshifters, fiction, paranormal,witches
Rating:  4.22
Distance: 0.7459

Result 4
Title:   Magic's Promise (Valdemar: Last Herald-Mage #2)
Author:  Mercedes Lackey
Genres:  fantasy, fiction, lgbt, f

In [6]:
from src.retriever.retriever import Retriever

retriever = Retriever(
    index_path="../models/faiss_index.bin",
    metadata_path="../models/metadata.pkl",
    model_name="all-MiniLM-L6-v2",
    k=5
)

query = "epic fantasy with dragons and magic"
results = retriever.retrieve(query)

for i, r in enumerate(results, 1):
    meta = r["metadata"]
    print(f"\nResult {i}")
    print(f"Title: {meta['title']}")
    print(f"Author: {meta['author']}")
    print(f"Rating: {meta['rating']}")
    print(f"Score: {r['score']:.3f}")

Loading embedding model: all-MiniLM-L6-v2
Loaded index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)

Result 1
Title: Master of Dragons (Mageverse #5)
Author: Angela Knight
Rating: 4.22
Score: 0.620

Result 2
Title: A Practical Guide to Dragons
Author: Lisa Trumbauer/Emily Fiegenshuh
Rating: 4.33
Score: 0.602

Result 3
Title: The Girl the Dragon and the Wild Magic (Rhianna #1)
Author: Dave Luckett
Rating: 3.86
Score: 0.600

Result 4
Title: The Dragon Quintet
Author: Marvin Kaye/Michael Swanwick/Orson Scott Card/Mercedes Lackey/Tanith Lee/Elizabeth Moon
Rating: 3.77
Score: 0.579

Result 5
Title: The Magic of Krynn (DragonLance: Tales I #1)
Author: Margaret Weis/Tracy Hickman/Barbara Siegel/Scott Siegel/Roger E. Moore/Warren B. Smith/Nick O'Donohoe/Richard A. Knaak/Nancy Varian Berberick/Mary L. Kirchoff/Michael Williams
Rating: 3.74
Score: 0.577


In [7]:
from src.service.book_recommendation_service import BookRecommendationService

service = BookRecommendationService(
    index_path="../models/faiss_index.bin",
    metadata_path="../models/metadata.pkl",
    model_name="all-MiniLM-L6-v2",
    default_k=5,
)

query = "light-hearted romance books with magic"

recs = service.recommend_books(
    query=query,
    n_results=5,
    min_rating=3.5,
    genre_filter=None,
)

for i, b in enumerate(recs, start=1):
    print(f"\nResult {i}")
    print(f"Title: {b['title']}")
    print(f"Author: {b['author']}")
    print(f"Rating: {b['rating']}")
    print(f"Year: {b['year']}")
    print(f"Genres: {b['genres']}")
    print(f"Score: {b['score']:.3f}")

Loading embedding model: all-MiniLM-L6-v2
Loaded index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)

Result 1
Title: Magic's Promise (Valdemar: Last Herald-Mage #2)
Author: Mercedes Lackey
Rating: 4.21
Year: 1990
Genres: fantasy, fiction, lgbt, fantasy,magic, science fiction fantasy, fantasy,high fantasy, romance,m m romance, young adult, lgbt,queer, romance
Score: 0.605

Result 2
Title: Magic's Pawn (Valdemar: Last Herald-Mage #1)
Author: Mercedes Lackey
Rating: 4.21
Year: 1989
Genres: fantasy, fiction, lgbt, fantasy,magic, romance, romance,m m romance, fantasy,high fantasy, science fiction fantasy, young adult, lgbt,queer
Score: 0.591

Result 3
Title: Magic's Price (Valdemar: Last Herald-Mage #3)
Author: Mercedes Lackey
Rating: 4.27
Year: 1990
Genres: fantasy, fiction, lgbt, fantasy,magic, science fiction fantasy, romance, fantasy,high fantasy, romance,m m romance, young adult, fantasy,epic fantasy
Score: 0.587

Result 4
Title: The Books of Mag

In [9]:
from src.service.book_recommendation_service import BookRecommendationService
from src.generator.rag_generator import RAGGenerator

# Attach fake generator for now
service = BookRecommendationService(
    index_path="../models/faiss_index.bin",
    metadata_path="../models/metadata.pkl",
    model_name="all-MiniLM-L6-v2",
    default_k=5,
    generator=RAGGenerator(model="lamma3")
)

query = "fantasy books with dragons and romance"
result = service.pipeline.run(query)

print(result["answer"])


Loading embedding model: all-MiniLM-L6-v2
Loaded index from ../models/faiss_index.bin
Loaded metadata from ../models/metadata.pkl (n=10710)
Here are some books you might like based on your request:

1. A Practical Guide to Dragons by Lisa Trumbauer/Emily Fiegenshuh — This matches because it has themes related to your query.
2. The Book of the Dragon by Ciruelo Cabral — This matches because it has themes related to your query.
3. The Dragon Quintet by Marvin Kaye/Michael Swanwick/Orson Scott Card/Mercedes Lackey/Tanith Lee/Elizabeth Moon — This matches because it has themes related to your query.
